<a href="https://colab.research.google.com/github/dsfsgroup4/customer-sentiment-analysis/blob/main/embeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from numpy.linalg import norm

In [99]:
from google.colab import drive

# Monter Google Drive
drive.mount('/content/drive')
!ls '/content/drive/My Drive/Colab Notebooks/Jedha/Projet/data/'
path = "/content/drive/My Drive/Colab Notebooks/Jedha/Projet/data/roberta_results.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
data_avec_labels.csv  model_roberta_results.csv  model_roberta_results.csv.csv	roberta_results.csv


In [96]:
df1= pd.read_csv("/content/drive/My Drive/Colab Notebooks/Jedha/Projet/data/roberta_results.csv", encoding='latin-1')

In [97]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21812 entries, 0 to 21811
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        21812 non-null  int64  
 1   reviewer_id       21812 non-null  int64  
 2   store_address     21812 non-null  object 
 3   latitude          21812 non-null  float64
 4   longitude         21812 non-null  float64
 5   review_time       21812 non-null  object 
 6   review            21812 non-null  object 
 7   rating            21812 non-null  object 
 8   review_date       21812 non-null  object 
 9   City              21812 non-null  object 
 10  State             21812 non-null  object 
 11  ZipCode           21812 non-null  int64  
 12  rating_int        21812 non-null  int64  
 13  clean_reviews     21811 non-null  object 
 14  actual_sentiment  21812 non-null  object 
 15  pred_sentiment    21811 non-null  object 
 16  RoBERTa_score     21811 non-null  float6

In [100]:
df = pd.read_csv(path, encoding='latin-1')

In [101]:
df.columns

Index(['Unnamed: 0', 'reviewer_id', 'store_address', 'latitude', 'longitude',
       'review_time', 'review', 'rating', 'review_date', 'City', 'State',
       'ZipCode', 'rating_int', 'clean_reviews', 'actual_sentiment',
       'pred_sentiment', 'RoBERTa_score'],
      dtype='object')

In [102]:
df.head()

,Unnamed: 0,reviewer_id,store_address,latitude,longitude,review_time,review,rating,review_date,City,State,ZipCode,rating_int,clean_reviews,actual_sentiment,pred_sentiment,RoBERTa_score
0,0,1,13749 US-183 Hwy,30.460718,-97.792874,3 months ago,"Why does it look like someone spit on my food?\nI had a normal transaction, everyone was chill ...",1 star,2025-01-08,Austin,TX,78750,1,Why does it look like someone spit on my food I had a normal transaction everyone was chill and ...,negative,negative,0.917278
1,1,2,13749 US-183 Hwy,30.460718,-97.792874,5 days ago,It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make ...,4 stars,2025-04-03,Austin,TX,78750,4,Itd McDonalds It is what it is as far as the food and atmosphere go The staff here does make a d...,positive,positive,0.972301
2,2,3,13749 US-183 Hwy,30.460718,-97.792874,5 days ago,Made a mobile order got to the speaker and checked it in.\nLine was not moving so I had to leave...,1 star,2025-04-03,Austin,TX,78750,1,Made a mobile order got to the speaker and checked it in Line was not moving so I had to leave o...,negative,negative,0.661857
3,3,4,13749 US-183 Hwy,30.460718,-97.792874,a month ago,My mc. Crispy chicken sandwich was .................................... customer service was qui...,5 stars,2025-03-08,Austin,TX,78750,5,My mc Crispy chicken sandwich was customer service was quick and p,positive,positive,0.940411
4,4,5,13749 US-183 Hwy,30.460718,-97.792874,2 months ago,"I repeat my order 3 times in the drive thru, and she still manage to mess it up , it was suppose...",1 star,2025-02-08,Austin,TX,78750,1,I repeat my order times in the drive thru and she still manage to mess it up it was suppose to b...,negative,negative,0.913123


In [103]:
#pour vérifier que la limite de token du model est suffisante
print("longeure moyenne :" , df['review'].apply(len).mean())
print("longeure mediane :" , df['review'].apply(len).median())
print("longeure max :" , df['review'].apply(len).max())
print("longeure min :" , df['review'].apply(len).min())

longeure moyenne : 142.5908215661104
longeure mediane : 83.0
longeure max : 2553
longeure min : 1


# Embedding des labels

In [130]:
labels = [
    'hygiene', 'food quality', 'food', 'staff', 'something is missing',
    'location', 'speed of service', 'drive-thru', 'temperature of the food',
    'atmosphere', 'customer service', "temperature" , "price", "speed", "quality", "courtesy",
]

In [105]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') #embeding à 384 dimentions, max token = 256
labels_embedded = model.encode(labels)

# test du l'embedin et dot product sur un seul restaurant

In [ ]:
df["store_address"][0]

'13749 US-183 Hwy, Austin, TX 78750, United States'

In [ ]:
#selection du premier restaurant
df_1restaurant = df[df["store_address"]=="13749 US-183 Hwy, Austin, TX 78750, United States"]
df_1restaurant.shape

(578, 16)

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') #embeding à 384 dimentions, max token = 256
reviews_embedded = model.encode(df_1restaurant["review"])
df_1restaurant['review_embedded'] = reviews_embedded.tolist()

<ipython-input-194-6e20b6632105>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1restaurant['review_embedded'] = reviews_embedded.tolist()


In [ ]:
df_1restaurant["review_embedded"].shape

(578,)

In [ ]:
df_1restaurant['review_embedded'].head()

,review_embedded
0,"[0.022007549181580544, -0.048939093947410583, 0.03766841068863869, -0.0076949819922447205, 0.025..."
1,"[0.03945782035589218, -0.015777520835399628, 0.07758671045303345, 0.013737925328314304, 0.014259..."
2,"[-0.05103641003370285, 0.04763896018266678, 0.05201364681124687, -0.04680230841040611, 0.0035369..."
3,"[-0.04021293297410011, -0.024235272780060768, 0.05131354555487633, -0.009905711747705936, -0.025..."
4,"[0.04103447124361992, 0.006413918454200029, 0.12267220765352249, 0.021540647372603416, -0.102571..."


In [ ]:
# Calculer le produit scalaire normalisé pour chaque label et trouver le label avec le score le plus élevé
top_labels = []
for review in reviews_embedded:
    label_scores = [
        (label, np.dot(review, labels_embedded[i]) / (norm(review) * norm(labels_embedded[i])))
        for i, label in enumerate(labels)
    ]
    # Trier les labels par score décroissant
    sorted_labels = sorted(label_scores, key=lambda x: x[1], reverse=True)
    # Sélectionner les cinq premiers labels
    top_labels.append(sorted_labels[:5])

# Ajouter des colonnes pour les cinq labels ayant les scores les plus élevés
for i in range(5):
    df_1restaurant[f'top_{i+1}_label'] = [labels[i] for labels in top_labels]

# Augmenter la limite de caractères affichés par cellule
pd.set_option('display.max_colwidth', 100)

df_1restaurant[["review",\
                'top_1_label', 'top_2_label','top_3_label', 'top_4_label', 'top_5_label']].head(10)



<ipython-input-197-d9e6b03c5bf1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1restaurant[f'top_{i+1}_label'] = [labels[i] for labels in top_labels]
<ipython-input-197-d9e6b03c5bf1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1restaurant[f'top_{i+1}_label'] = [labels[i] for labels in top_labels]
<ipython-input-197-d9e6b03c5bf1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

,review,top_1_label,top_2_label,top_3_label,top_4_label,top_5_label
0,"Why does it look like someone spit on my food?\nI had a normal transaction, everyone was chill ...","(hygiene, 0.28746822)","(food, 0.27422136)","(temperature of the food, 0.25352862)","(food quality, 0.19775568)","(drive-thru, 0.17217408)"
1,It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make ...,"(staff, 0.3886896)","(food quality, 0.36147067)","(customer service, 0.30244836)","(food, 0.2999506)","(temperature of the food, 0.226313)"
2,Made a mobile order got to the speaker and checked it in.\nLine was not moving so I had to leave...,"(customer service, 0.3827942)","(staff, 0.14755319)","(drive-thru, 0.14341448)","(something is missing, 0.14244273)","(location, 0.14115314)"
3,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï...,"(food quality, 0.42417273)","(temperature of the food, 0.35612532)","(food, 0.33923125)","(customer service, 0.296566)","(speed of service, 0.19602737)"
4,"I repeat my order 3 times in the drive thru, and she still manage to mess it up , it was suppose...","(drive-thru, 0.40096334)","(food, 0.26292616)","(food quality, 0.22811927)","(customer service, 0.1991002)","(temperature of the food, 0.18270876)"
5,I work for door dash and they locked us all out to wait in a long line for no reason at 10. I tr...,"(customer service, 0.2676896)","(drive-thru, 0.1942678)","(staff, 0.16171806)","(location, 0.14941879)","(speed of service, 0.13265584)"
6,"If I could give this location a zero on customer service, I would. Pulled into drive through a...","(customer service, 0.3626177)","(drive-thru, 0.24972373)","(food quality, 0.18007174)","(staff, 0.17227705)","(speed of service, 0.17197515)"
7,Came in and ordered a Large coffee w/no ice. They handed me a cup that was less than half way fu...,"(drive-thru, 0.1847711)","(customer service, 0.1640311)","(temperature of the food, 0.14997266)","(food quality, 0.11158571)","(staff, 0.10174852)"
8,Went thru drive thru. Ordered. Getting home noticed my 10 piece nuggets where missing. Tried cal...,"(something is missing, 0.31157857)","(food, 0.29818207)","(food quality, 0.28817856)","(drive-thru, 0.28361377)","(temperature of the food, 0.26132107)"
9,"I'm not really a huge fan of fast food, but I have 2 teenage daughter's who enjoy McDs. The staf...","(food quality, 0.38792324)","(customer service, 0.29408354)","(drive-thru, 0.26169395)","(speed of service, 0.25601938)","(food, 0.22585614)"


In [ ]:
df_1restaurant.columns

Index(['Unnamed: 0', 'reviewer_id', 'pred_sentiment', 'RoBERTa_score',
       'clean_reviews', 'rating', 'actual_sentiment', 'correct_prediction',
       'latitude ', 'longitude', 'review_date', 'City', 'State', 'review',
       'store_address', 'review_time', 'review_embedded', 'top_1_label',
       'top_2_label', 'top_3_label', 'top_4_label', 'top_5_label'],
      dtype='object')

# Embedding et dot product sur tout le dataset

In [106]:
from tqdm.notebook import tqdm

# Remplacer les valeurs manquantes par une chaîne vide
df['clean_reviews'] = df['clean_reviews'].fillna('')

# Charger le modèle SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Encoder les avis avec une barre de progression
reviews_embedded = []
for review in tqdm(df["clean_reviews"], desc="Encoding reviews"):
    reviews_embedded.append(model.encode(review))

# Ajouter les embeddings au DataFrame
df['review_embedded'] = reviews_embedded

Encoding reviews:   0%|          | 0/21812 [00:00<?, ?it/s]

In [110]:
df.to_csv("/content/drive/My Drive/Colab Notebooks/Jedha/Projet/data/data_avec_labels.csv", index=False)
df= pd.read_csv("/content/drive/My Drive/Colab Notebooks/Jedha/Projet/data/data_avec_labels.csv", encoding='latin-1')

In [111]:
df.columns

Index(['Unnamed: 0', 'reviewer_id', 'store_address', 'latitude', 'longitude',
       'review_time', 'review', 'rating', 'review_date', 'City', 'State',
       'ZipCode', 'rating_int', 'clean_reviews', 'actual_sentiment',
       'pred_sentiment', 'RoBERTa_score', 'review_embedded'],
      dtype='object')

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21812 entries, 0 to 21811
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        21812 non-null  int64  
 1   reviewer_id       21812 non-null  int64  
 2   store_address     21812 non-null  object 
 3   latitude          21812 non-null  float64
 4   longitude         21812 non-null  float64
 5   review_time       21812 non-null  object 
 6   review            21812 non-null  object 
 7   rating            21812 non-null  object 
 8   review_date       21812 non-null  object 
 9   City              21812 non-null  object 
 10  State             21812 non-null  object 
 11  ZipCode           21812 non-null  int64  
 12  rating_int        21812 non-null  int64  
 13  clean_reviews     21811 non-null  object 
 14  actual_sentiment  21812 non-null  object 
 15  pred_sentiment    21811 non-null  object 
 16  RoBERTa_score     21811 non-null  float6

In [113]:
# Calculer le score pour chaque label et ajouter une colonne pour chaque label
for i, label in enumerate(labels):
    df[label] = [
        np.dot(review, labels_embedded[i]) / (norm(review) * norm(labels_embedded[i]))
        for review in reviews_embedded
    ]
df.head()

,Unnamed: 0,reviewer_id,store_address,latitude,longitude,review_time,review,rating,review_date,City,...,speed of service,drive-thru,temperature of the food,atmosphere,customer service,temperature,price,speed,quality,courtesy
0,0,1,13749 US-183 Hwy,30.460718,-97.792874,3 months ago,"Why does it look like someone spit on my food?\nI had a normal transaction, everyone was chill ...",1 star,2025-01-08,Austin,...,0.001561,0.158012,0.253002,0.078995,0.083363,0.072332,0.068902,0.065940,0.047876,0.193300
1,1,2,13749 US-183 Hwy,30.460718,-97.792874,5 days ago,It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make ...,4 stars,2025-04-03,Austin,...,0.217116,0.213600,0.235496,0.067040,0.264076,0.081645,-0.018591,0.081617,0.184792,0.150819
2,2,3,13749 US-183 Hwy,30.460718,-97.792874,5 days ago,Made a mobile order got to the speaker and checked it in.\nLine was not moving so I had to leave...,1 star,2025-04-03,Austin,...,0.127838,0.141085,0.007825,0.013562,0.376883,-0.050720,0.073238,0.034701,0.075152,0.157303
3,3,4,13749 US-183 Hwy,30.460718,-97.792874,a month ago,My mc. Crispy chicken sandwich was .................................... customer service was qui...,5 stars,2025-03-08,Austin,...,0.374259,0.204817,0.246718,0.050981,0.464530,0.037117,0.154002,0.232614,0.263690,0.200330
4,4,5,13749 US-183 Hwy,30.460718,-97.792874,2 months ago,"I repeat my order 3 times in the drive thru, and she still manage to mess it up , it was suppose...",1 star,2025-02-08,Austin,...,0.159322,0.394592,0.178700,-0.095313,0.200030,-0.068990,-0.052336,0.114158,0.127601,0.262265


In [114]:
df.columns

Index(['Unnamed: 0', 'reviewer_id', 'store_address', 'latitude', 'longitude',
       'review_time', 'review', 'rating', 'review_date', 'City', 'State',
       'ZipCode', 'rating_int', 'clean_reviews', 'actual_sentiment',
       'pred_sentiment', 'RoBERTa_score', 'review_embedded', 'hygiene',
       'food quality', 'food', 'staff', 'something is missing', 'location',
       'speed of service', 'drive-thru', 'temperature of the food',
       'atmosphere', 'customer service', 'temperature', 'price', 'speed',
       'quality', 'courtesy'],
      dtype='object')

In [115]:
df=df[['reviewer_id',
       'pred_sentiment', 'RoBERTa_score',
       'review','clean_reviews',
       'hygiene', 'food quality', 'food', 'staff', 'something is missing', 'location','speed of service', 'drive-thru', 'temperature of the food','atmosphere', 'customer service', 'temperature', 'price', 'speed','quality', 'courtesy',
       'review_time', 'review_date',
       'store_address', 'latitude', 'longitude','City', 'State',
       'rating_int', 'actual_sentiment' ]]
df.head()

,reviewer_id,pred_sentiment,RoBERTa_score,review,clean_reviews,hygiene,food quality,food,staff,something is missing,...,courtesy,review_time,review_date,store_address,latitude,longitude,City,State,rating_int,actual_sentiment
0,1,negative,0.917278,"Why does it look like someone spit on my food?\nI had a normal transaction, everyone was chill ...",Why does it look like someone spit on my food I had a normal transaction everyone was chill and ...,0.295526,0.199891,0.290295,0.066378,0.170541,...,0.193300,3 months ago,2025-01-08,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,1,negative
1,2,positive,0.972301,It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make ...,Itd McDonalds It is what it is as far as the food and atmosphere go The staff here does make a d...,0.198294,0.449936,0.266937,0.283245,0.058948,...,0.150819,5 days ago,2025-04-03,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,4,positive
2,3,negative,0.661857,Made a mobile order got to the speaker and checked it in.\nLine was not moving so I had to leave...,Made a mobile order got to the speaker and checked it in Line was not moving so I had to leave o...,-0.056594,-0.019245,0.047143,0.112900,0.112442,...,0.157303,5 days ago,2025-04-03,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,1,negative
3,4,positive,0.940411,My mc. Crispy chicken sandwich was .................................... customer service was qui...,My mc Crispy chicken sandwich was customer service was quick and p,0.101898,0.349108,0.291640,0.172535,0.071012,...,0.200330,a month ago,2025-03-08,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,5,positive
4,5,negative,0.913123,"I repeat my order 3 times in the drive thru, and she still manage to mess it up , it was suppose...",I repeat my order times in the drive thru and she still manage to mess it up it was suppose to b...,0.123227,0.231226,0.257628,0.165260,0.146708,...,0.262265,2 months ago,2025-02-08,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,1,negative


In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21812 entries, 0 to 21811
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   reviewer_id              21812 non-null  int64  
 1   pred_sentiment           21811 non-null  object 
 2   RoBERTa_score            21811 non-null  float64
 3   review                   21812 non-null  object 
 4   clean_reviews            21811 non-null  object 
 5   hygiene                  21812 non-null  float32
 6   food quality             21812 non-null  float32
 7   food                     21812 non-null  float32
 8   staff                    21812 non-null  float32
 9   something is missing     21812 non-null  float32
 10  location                 21812 non-null  float32
 11  speed of service         21812 non-null  float32
 12  drive-thru               21812 non-null  float32
 13  temperature of the food  21812 non-null  float32
 14  atmosphere            

In [117]:
df.shape

(21812, 30)

In [118]:
df.to_csv("/content/drive/My Drive/Colab Notebooks/Jedha/Projet/data/data_avec_labels.csv", index=False)

In [125]:
# Convertir les colonnes de labels en numériques
for label in df.columns[5:21]:
    df[label] = pd.to_numeric(df[label], errors='coerce')

In [127]:
df.to_csv("/content/drive/My Drive/Colab Notebooks/Jedha/Projet/data/data_avec_labels.csv", index=False)

In [129]:
contains_str = df.applymap(lambda x: isinstance(x, str)).any()
contains_str[contains_str==True]

<ipython-input-129-18e0ead50aa1>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_str = df.applymap(lambda x: isinstance(x, str)).any()


,0
pred_sentiment,True
review,True
clean_reviews,True
review_time,True
review_date,True
store_address,True
City,True
State,True
actual_sentiment,True
